In [99]:
import json
import os
import pdb

import inro.emme.datatable as _dt
import inro.emme.desktop.worksheet as _ws
import inro.emme.network as _nw
import pandas as _pd

mm = inro.modeller.Modeller()
desktop = mm.desktop

In [100]:
# modeller tools used
create_attribute = mm.tool('inro.emme.data.extra_attribute.create_extra_attribute')

In [101]:
# names of attributes used (existing and to be created)

x_alight = '@alight_per_door'
x_board = '@board_per_door'
y_dwt = '@dwtout'
f_crowd = '@crowding_factor'

dwt_in = '@dwtin'

In [102]:
# import average dwell times and 
project_path = os.path.dirname(desktop.project.path)
df_line_taz = _pd.read_csv(os.path.join(project_path, 'Media','dwell_byLineTAZ.csv'))
df_taz = _pd.read_csv(os.path.join(project_path, 'Media','dwell_byTAZ.csv'))

In [103]:
# create data tables of transit segment attributes and taz, join tables

# TODO: convert this cell into external module

project_path = os.path.dirname(desktop.project.path)
project_table_db = desktop.project.data_tables()

### TAZs
# import TAZ shp

taz_path = os.path.join(project_path, 'Scripts\MX_Task20\GGHMv4_TAZ_updated\GGHMv4_TAZ_updated.shp').replace("\\", "/")
taz_ds = _dt.DataSource(taz_path)
taz_data = taz_ds.layer('GGHMv4_TAZ_updated').get_data()

# remove all attributes except TAZ_NO
att_delete = []
taz_att = taz_data.attributes()
for att in taz_att:
    if att.name != 'TAZ_NO' and att.atype != 'GEOMETRY':
        att_delete.append(att.name)

for att in att_delete:
    taz_data.delete_attribute(att)

# create table out of data
project_table_db.create_table('TAZ_shp', taz_data, overwrite = True)


### Transit Segment
# create transit segment data table from network table

root_worksheet_folder = desktop.root_worksheet_folder()
tseg_table = root_worksheet_folder.find_item(["General", "Network", "Transit", 'Transit segments'])

# remove unnecessary columns
tseg = tseg_table.open()
for i in range(8):
    tseg.delete_column(1)

# create new column for segment id
id_col = _ws.Column()
id_col.name = 'id'
id_col.expression = 'id'
tseg.add_column(0, id_col)

tseg.par('Filter').set('')

# save sheet
tseg.par('Name').set('TransitSegments_noDwell')
tseg.save_as_data_table('TransitSegments_noDwell', overwrite = True)
tseg.close()

# gets the TAZ and segment data tables, joins TAZ to segments spatially
project_table_db = desktop.project.data_tables()
taz_dt = project_table_db.table('TAZ_shp')
dt_attributes(taz_dt)
taz_df = to_pandas(taz_data)
seg_dt = project_table_db.table('TransitSegments_noDwell')
# seg_df = to_pandas(seg_dt.get_data())
joint_dt = seg_dt.spatial_join(taz_dt,
                    predicate = 'INTERSECT')

Attribute(TAZ_NO, REAL)

None

GeometryAttribute(geometry, POLYGON)

None

In [104]:
# convert joined table to data frame

# data table to pandas data frame
def to_pandas(dat):
    dfs = []
    for a in dat.attributes():
        if a.atype == 'GEOMETRY':
            dfs.append( _pd.DataFrame( {a.name: [x.text for x in a.values]} ) )
        else:
            dfs.append( _pd.DataFrame( {a.name: a.values} ) )
    df = dfs[0]
    for x in dfs[1:]:
        df = df.join(x)
    return df

df = to_pandas(joint_dt)
df = df[['Line','id','TAZ_NO']] # filter to needed columns
# df.set_index(['id'], inplace = True)

In [105]:
# seg_df = to_pandas(seg_dt)
# taz_df = to_pandas(taz_dt)
def dt_attributes(dt):
    from IPython.display import display
    for att in dt.attributes():
        display(dt.attribute(att.name))
        display(dt.attribute(att.name).values)
    return


In [106]:
# apply imported average dwell times and merge with segment data
# pdb.set_trace()
# join with imported average dwell times
# a.reset_index().merge(b, how="left").set_index('index')
joint_df = df.merge(df_line_taz,
                    how = 'left',
                    left_on = ['Line','TAZ_NO'],
                    right_on = ['Line','TAZ_NO']
                    )
# pdb.set_trace()
joint_df.rename(columns = {'modelled_dwell':'dwell_lt'}, inplace = True)
joint_df = joint_df.merge(df_taz,
                    how = 'left',
                    left_on = ['TAZ_NO'],
                    right_on = ['TAZ_NO']
                          )
joint_df.rename(columns = {'modelled_dwell':'dwell_t'}, inplace = True)
# joint_df.set_index('id')
# pdb.set_trace()

In [107]:
# create attribute for imported dwell times

# function to create extra attributes for transit segments
def create_att_segment( att, desc):
    create_attribute(extra_attribute_type='TRANSIT_SEGMENT',
                 extra_attribute_name=att,
                 extra_attribute_description=desc,
                 overwrite=True)
    return

# attributes to create and their descriptions
new_atts = {dwt_in:'imported dwt'}

#create the attributes
for att in new_atts:
    create_att_segment(att, new_atts[att])
    
# get network to apply dwell times

scen = mm.emmebank.scenario(100)
network = scen.get_network()
# network.publishable = True


In [109]:
# iterate through network elements and apply dwell times
from time import time
import numpy as _np
joint_df_avg = joint_df.groupby('id').mean()

t0 = time()
for seg in network.transit_segments():
    if seg.j_node != None:
        seg_id = seg.id
        seg_dwell_lt = joint_df_avg.loc[seg_id, 'dwell_lt']
        seg_dwell_t = joint_df_avg.loc[seg_id, 'dwell_t']
    #     seg_dwell_lt = 0
    #     seg_dwell_t = 0
        if not _np.isnan(seg_dwell_lt):
#             display('lt')
#             print(seg_id + ' has line and TAZ match')
            seg['@dwtin'] = seg_dwell_lt
        elif not _np.isnan(seg_dwell_t):
#             display( 'Used TAZ dwt for ' + seg_id )
#             print(seg_id + ' only has TAZ match')
            seg['@dwtin'] = seg_dwell_t
        else:
#             print(seg_id + ' has no match. Dwell time set to 0.')
            seg['@dwtin'] = 0
#         print type(seg_dwell_lt)
t1 = time()
rt = (t1-t0)/60
print 'Runtime: {}'.format(rt)

In [110]:
with open('specs.json') as specs:
    vehicle_grouping = json.load(specs)['vehicle_grouping']

def get_veh_filter(vehicle_types):
    veh_list = []
    for veh in vehicle_types:
        veh_list.append('vehicle = ' + str(veh))
        veh_str = ' | '.join(veh_list)
    return veh_str, veh_list

total_veh_list = []
for group in vehicle_grouping:
    vehicle_types = vehicle_grouping[group]['vehicle types']
    # expression string for vehicle types
    veh_str, veh_list = get_veh_filter(vehicle_types)
    total_veh_list.extend(veh_list)
line_filter = ' | '.join(total_veh_list)
    

In [111]:
print line_filter

In [112]:
scen.publish_network(network)

In [113]:
# joint_df.to_csv(os.path.join(project_path, 'Media','applied_dwt.csv'))
# df.to_csv(os.path.join(project_path, 'Media','seg_taz.csv'))
seg_df = to_pandas(seg_dt.get_data())
seg_df.to_csv(os.path.join(project_path, 'Media','seg_df.csv'))
taz_df = to_pandas(taz_dt.get_data())
taz_df.to_csv(os.path.join(project_path, 'Media','taz_df.csv'))